In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf
from plotly.colors import n_colors
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-24 12:19:05.012569: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-24 12:19:05.070396: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-24 12:19:05.344797: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-24 12:19:05.344911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-24 12:19:05.392785: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
from scipy.optimize import basinhopping, minimize
from tensorflow.keras.losses import MSE

In [3]:
tree = Node.from_sequence(
    [
        BASE_TOKENS.symbols.index(value)
        for value in ["sin", "+", "*", "const", "var_x", "const"]
    ]
)

X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

# TF

In [4]:
import functools

import tensorflow_probability as tfp
from tensorflow.keras.losses import MSE

In [6]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())
model = Expression(tree)

In [14]:
model.constants = np.array([0.5, 0], dtype=np.float32)

In [15]:
model({"X": X, "constants": np.random.randn(2)})

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([-0.36892688, -0.258867  , -0.14532782, -0.02983494,  0.08605924,
        0.20079611,  0.3128341 ,  0.420667  ,  0.52284527,  0.6179954 ,
        0.70483834,  0.78220683,  0.8490609 ,  0.9045018 ,  0.94778436,
        0.9783267 ,  0.9957183 ,  0.99972546,  0.9902942 ,  0.9675514 ,
        0.93180263,  0.88352865,  0.82337815,  0.7521598 ,  0.6708307 ,
        0.58048445,  0.48233518,  0.37770241,  0.26799253,  0.15468028,
        0.03928872, -0.07663085, -0.19152041, -0.30383554, -0.4120665 ,
       -0.5147583 , -0.6105308 , -0.6980965 , -0.7762785 , -0.8440255 ,
       -0.9004271 , -0.94472516, -0.97632414, -0.9947993 , -0.99990237,
       -0.9915647 , -0.9698982 , -0.9351944 , -0.8879197 , -0.82870936],
      dtype=float32)>

In [18]:
model.optimize_constants(X, y, "lbfgs")

In [41]:
def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_sum(tf.pow(y_true - y_pred, 2)) / y_true.shape[0])


def normalized_rmse(y_true, y_pred):
    return tf.math.reduce_std(y_true) * RMSE(y_true, y_pred)


def reward(y_true, y_pred):
    return 1 / (1 + normalized_rmse(y_true, y_pred))

In [36]:
y_true = tf.convert_to_tensor(y, dtype=tf.float32)

In [42]:
reward(y_true, model.eval(X))

<tf.Tensor: shape=(), dtype=float32, numpy=0.9999999>

# Expression

In [ ]:
expression = Expression(tree)

In [ ]:
results = {}
T = 1e-1
step_size = 1
for n in range(50):
    res = basinhopping(
        lambda constants: MSE(y, expression.eval(X, constants)),
        expression.constants,
        T=T,
        stepsize=step_size,
        niter=1000,
        # niter_success=50
    )
    results[(T, step_size, n)] = res.x

results = pd.Series(results).rename_axis(["T", "step_size", "n"]).to_frame("res")
results.reset_index(inplace=True)
results["mse"] = results.apply(
    lambda x: MSE(y, expression.eval(X, x.res)).numpy(), axis=1
)

In [ ]:
(results.mse < 1e-3).sum()

In [ ]:
results = {}
for step_size in np.logspace(-1, 1, 10):
    for T in np.logspace(-3, 3, 10):
        for n in range(30):
            res = basinhopping(
                lambda constants: MSE(y, expression.eval(X, constants)),
                expression.constants,
                T=T,
                stepsize=step_size,
                niter=500,
            )
            results[(T, step_size, n)] = res.x

results = pd.Series(results).rename_axis(["T", "step_size", "n"]).to_frame("res")
results.reset_index(inplace=True)
results["mse"] = results.apply(
    lambda x: MSE(y, expression.eval(X, x.res)).numpy(), axis=1
)

In [ ]:
px.imshow(
    results.groupby(["T", "step_size"]).apply(lambda x: (x.mse < 1e-1).sum()).unstack(),
    aspect="auto",
).update_layout(xaxis_type="log", yaxis_type="log")

In [ ]:
fig = go.Figure()

colors = n_colors(
    "rgb(5, 200, 200)", "rgb(200, 10, 10)", results["T"].nunique(), colortype="rgb"
)
for (T, group), color in zip(results.groupby("T"), colors):
    fig.add_violin(x=group.distance, name=f"{T:.03f}", line_color=color)
fig.update_traces(orientation="h", side="positive", points=False)
fig.update_layout(height=600)